# Import dependencies

In [1]:
import os
os.environ['DDE_BACKEND'] = "pytorch"

import deepxde as dde
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import pennylane as qml

from pathlib import Path

import pyvista as pv
pv.set_jupyter_backend("static")

device = torch.device("cuda")

Using backend: pytorch



In [2]:
def show_solution(func):
    # Input: scalar "u" to be displayed
    grid = pv.UniformGrid(dimensions=(150, 200, 1),
                          origin=(-0.5, -0.5, 0),
                          spacing=(0.01, 0.01, 0.01))
    
    
    X = grid.points[:, 0:2]
    if isinstance(func, dde.Model) or isinstance(func, dde.nn.NN):
        u = func.predict(X)[:, 0]
    else:
        u = func(X)
        
    grid.point_data["u"] = u.flatten(order="F")
    
    plotter = pv.Plotter()
    plotter.add_mesh(grid)
    plotter.view_xy()
    plotter.show()

# Define quantum model

In [3]:
n_qubits = 4
n_layers = 1
n_depth = 1 + 1

dev = qml.device("lightning.qubit", wires=n_qubits)

@qml.qnode(dev, interface="torch", diff_method="adjoint")
def circuit(inputs, weights):
    
    qml.StronglyEntanglingLayers(weights[0], wires=range(n_qubits))
    
    qml.RX(inputs[0], wires=0)
    qml.RX(inputs[0], wires=1)
    qml.RX(inputs[1], wires=2)
    qml.RX(inputs[1], wires=3)
    
    
    qml.StronglyEntanglingLayers(weights[1], wires=range(n_qubits))
    
    return [qml.expval(qml.PauliZ(0))]

weight_shapes = {"weights": (n_depth, n_layers, n_qubits, 3)}

print(qml.draw(circuit, expansion_strategy="device")(inputs=torch.arange(n_qubits),
                                                     weights=torch.rand(weight_shapes["weights"])))

0: ──Rot(0.49,0.77,0.38)─╭●───────╭X──RX(0.00)──Rot(0.16,0.77,0.53)─╭●───────╭X─┤  <Z>
1: ──Rot(0.15,0.18,0.34)─╰X─╭●────│───RX(0.00)──Rot(0.09,0.02,0.15)─╰X─╭●────│──┤     
2: ──Rot(0.73,0.24,0.85)────╰X─╭●─│───RX(1.00)──Rot(0.55,0.05,0.01)────╰X─╭●─│──┤     
3: ──Rot(0.43,0.08,0.96)───────╰X─╰●──RX(1.00)──Rot(0.39,0.19,0.33)───────╰X─╰●─┤     


In [4]:
class ParNet(dde.nn.pytorch.nn.NN):
    def __init__(self, circuit, weight_shapes):
        super().__init__()
        
        self.qlayers = nn.ModuleList([qml.qnn.TorchLayer(circuit, weight_shapes) for _ in range(3)])
        self.clayers = nn.ModuleList([nn.Sequential(nn.Linear(2, 10),
                                                    nn.ReLU(),
                                                    nn.Linear(10, 10),
                                                    nn.ReLU(),
                                                    nn.Linear(10, 1)) for _ in range(3)])
        self.addups = nn.ModuleList([nn.Linear(3, 1) for _ in range(3)])
 
        
        
    def forward(self, x):
        # input: (N, 2)
        # output: (N, 3)
        y = [None] * 3
        for i in range(3): # separate networks for (u, v, p)
            x_q = torch.clone(x)
            x_c = torch.clone(x)
            
            x_q = self.qlayers[i](x_q).view(-1, 1)
            x_c = self.clayers[i](x_c).view(-1, 1)
            
            y[i] = self.addups[i](torch.hstack([x_q * x_c, x_q, x_c])) # span{x_q * x_c, x_q, x_c} 
            
        return torch.hstack(y)

In [5]:
testnet = ParNet(circuit, weight_shapes)

In [6]:
testnet(torch.rand(5, 2))

/home/lurker/miniconda3/envs/torch_dde_q/lib/python3.9/site-packages/pennylane/tape/qscript.py:569: UserWarning: The get_operation will soon be updated to also return the index of the trainable operation in the tape. If you want to switch to the new behavior, you can pass `return_op_index=True`
  warnings.warn(


tensor([[ 0.5757,  0.2805, -0.0457],
        [ 0.5542,  0.2910, -0.0286],
        [ 0.5409,  0.3606, -0.0440],
        [ 0.5402,  0.3216, -0.0561],
        [ 0.5773,  0.2727, -0.0500]], grad_fn=<CatBackward0>)

# Define governing equations

In [7]:
Re = 20
nu = 1 / Re
l = 1 / (2 * nu) - np.sqrt(1 / (4 * nu ** 2) + 4 * np.pi ** 2)

In [8]:
def u_func(x):
    return 1 - np.exp(l * x[:, 0:1]) * np.cos(2 * np.pi * x[:, 1:2])


def v_func(x):
    return l / (2 * np.pi) * np.exp(l * x[:, 0:1]) * np.sin(2 * np.pi * x[:, 1:2])


def p_func(x):
    return 1 / 2 * (1 - np.exp(2 * l * x[:, 0:1]))

In [9]:
def pde(x, u):
    u_vel, v_vel, p = u[:, 0:1], u[:, 1:2], u[:, 2:]
    u_vel_x = dde.grad.jacobian(u, x, i=0, j=0)
    u_vel_y = dde.grad.jacobian(u, x, i=0, j=1)
    u_vel_xx = dde.grad.hessian(u, x, component=0, i=0, j=0)
    u_vel_yy = dde.grad.hessian(u, x, component=0, i=1, j=1)

    v_vel_x = dde.grad.jacobian(u, x, i=1, j=0)
    v_vel_y = dde.grad.jacobian(u, x, i=1, j=1)
    v_vel_xx = dde.grad.hessian(u, x, component=1, i=0, j=0)
    v_vel_yy = dde.grad.hessian(u, x, component=1, i=1, j=1)

    p_x = dde.grad.jacobian(u, x, i=2, j=0)
    p_y = dde.grad.jacobian(u, x, i=2, j=1)

    momentum_x = (
        u_vel * u_vel_x + v_vel * u_vel_y + p_x - 1 / Re * (u_vel_xx + u_vel_yy)
    )
    momentum_y = (
        u_vel * v_vel_x + v_vel * v_vel_y + p_y - 1 / Re * (v_vel_xx + v_vel_yy)
    )
    continuity = u_vel_x + v_vel_y

    return [momentum_x, momentum_y, continuity]

# Define BC

In [10]:
def boundary_outflow(x, on_boundary):
    return on_boundary and np.isclose(x[0], 1)

In [11]:
spatial_domain = dde.geometry.Rectangle(xmin=[-0.5, -0.5], xmax=[1, 1.5])

boundary_condition_u = dde.icbc.DirichletBC(
    spatial_domain, u_func, lambda _, on_boundary: on_boundary, component=0
)
boundary_condition_v = dde.icbc.DirichletBC(
    spatial_domain, v_func, lambda _, on_boundary: on_boundary, component=1
)
boundary_condition_right_p = dde.icbc.DirichletBC(
    spatial_domain, p_func, boundary_outflow, component=2
)

data = dde.data.PDE(
    spatial_domain,
    pde,
    [boundary_condition_u, boundary_condition_v, boundary_condition_right_p],
    num_domain=2601,
    num_boundary=400,
    num_test=None,
)

# Network

In [12]:
net = ParNet(circuit, weight_shapes)

In [13]:
model = dde.Model(data, net)

# Train Adam

In [14]:
ckpt_path = Path("models/onlyadam_q/ckpt")
ckpt_path.mkdir(parents=True, exist_ok=True)

loss_path = Path("models/onlyadam_q/loss")
loss_path.mkdir(parents=True, exist_ok=True)

In [ ]:
# train with adam
model.compile("adam", lr=1e-3)
checkpointer = dde.callbacks.ModelCheckpoint(ckpt_path / "ckpt",
                                             verbose=1,
                                             save_better_only=True)

losshistory, train_state = model.train(iterations=10, display_every=1, callbacks=[checkpointer])

Compiling model...
'compile' took 0.000482 s

Training model...

Step      Train loss                                                      Test loss                                                       Test metric
0         [8.59e-04, 2.91e-04, 3.53e-03, 1.99e+00, 1.24e-01, 9.77e-01]    [8.59e-04, 2.91e-04, 3.53e-03, 1.99e+00, 1.24e-01, 9.77e-01]    []  


In [1]:
dde.utils.external.save_loss_history(loss_history=losshistory, fname=loss_path / "loss")
dde.utils.external.plot_loss_history(loss_history=losshistory)

NameError: name 'dde' is not defined

# Train L-BFGS

In [ ]:
model.compile("L-BFGS")
losshistory, train_state = model.train()

# Loss & Inference

In [20]:
X = spatial_domain.uniform_points(100000)

output = model.predict(X)
u_pred = output[:, 0]
v_pred = output[:, 1]
p_pred = output[:, 2]

u_exact = u_func(X).reshape(-1)
v_exact = v_func(X).reshape(-1)
p_exact = p_func(X).reshape(-1)

f = model.predict(X, operator=pde)

l2_difference_u = dde.metrics.l2_relative_error(u_exact, u_pred)
l2_difference_v = dde.metrics.l2_relative_error(v_exact, v_pred)
l2_difference_p = dde.metrics.l2_relative_error(p_exact, p_pred)
residual = np.mean(np.absolute(f))

print("Mean residual:", residual)
print("L2 relative error in u:", l2_difference_u)
print("L2 relative error in v:", l2_difference_v)
print("L2 relative error in p:", l2_difference_p)

Mean residual: 0.0079963235
L2 relative error in u: 0.0046862364
L2 relative error in v: 0.019253293
L2 relative error in p: 0.006946357
